In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from datetime import datetime, date
from time import mktime
import mysql.connector
import pymysql
from sqlalchemy import create_engine

In [2]:

#date to start query
start = date(2012, 1,2)
unix_start = str(int(mktime(start.timetuple()))) +'/'

#date to end query
end = pd.to_datetime('today')
unix_end = str(int(mktime(end.timetuple()))) 

#api url
url = 'https://coinmetrics.io/api/v1/get_asset_data_for_time_range/'

#coin ticker
coin = 'btc/'

#data to be retreived
data_type_price = 'price(usd)/'
data_type_mcap = 'marketcap(usd)/'

#api call
coin_data = requests.get( url + coin+'/' + data_type_mcap + str(unix_start) + str(unix_end))

In [3]:
coin_list = ['btc','eth', 'eos']
#pd.options.display.int_format = '{:.9f}'.format
new_df = pd.DataFrame(None)

#converting json object
'''data = coin_data.json()
data = pd.DataFrame(data)
data = pd.DataFrame(data['result'].values.tolist(), columns=['Date','Price'])'''

for i in coin_list:

        coin_prices = requests.get( url + i+ '/' + data_type_price + str(unix_start) + str(unix_end))
        price_data = coin_prices.json()
        
        coin_mcap = requests.get( url + i+ '/' + data_type_mcap + str(unix_start) + str(unix_end))
        mcap_data = coin_mcap.json()
        
        price_df = pd.DataFrame(price_data)
        mcap_df = pd.DataFrame(mcap_data)
        
        price_df = pd.DataFrame(price_df['result'].values.tolist(), columns=['date',i+'_price'])
        mcap_df = pd.DataFrame(mcap_df['result'].values.tolist(), columns=['date',i+'_market_cap'])
        
        
        if new_df.empty:
            new_df = price_df.merge(mcap_df, on ="date")
        else:
            new_df = pd.merge(new_df, price_df, on ="date", how='outer')
            new_df = pd.merge(new_df, mcap_df, on ="date", how='outer')
            
new_df['date'] = pd.to_datetime(new_df['date'],unit='s')
new_df.fillna(0 , inplace=True)
new_df = new_df.set_index('date')
new_df.head()

,btc_price,btc_market_cap,eth_price,eth_market_cap,eos_price,eos_market_cap
date,,,,,,
2013-04-28,135.30,1.500656e+09,0.0,0.0,0.0,0.0
2013-04-29,134.44,1.491703e+09,0.0,0.0,0.0,0.0
2013-04-30,144.00,1.598310e+09,0.0,0.0,0.0,0.0
2013-05-01,139.00,1.543334e+09,0.0,0.0,0.0,0.0
2013-05-02,116.38,1.292577e+09,0.0,0.0,0.0,0.0


In [4]:
#create connection to mysql
host="localhost"
user="root"
passwd="password"
port = 3306
database = 'mydatabase'
engine = create_engine('mysql+pymysql://' + user + ':' + passwd + '@' + host + ':' + str(port) + '/' + database, echo=False)

#create sql table from dataframe
new_df.to_sql(name='cryptocurrency', con = engine, if_exists = 'append')



C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [5]:
new_df.tail()

,btc_price,btc_market_cap,eth_price,eth_market_cap,eos_price,eos_market_cap
date,,,,,,
2019-01-14,3557.31,6.218933e+10,116.98,1.220985e+10,2.26,2.047342e+09
2019-01-15,3704.22,6.476357e+10,129.17,1.348494e+10,2.48,2.250838e+09
2019-01-16,3631.51,6.349844e+10,121.81,1.271814e+10,2.40,2.172751e+09
2019-01-17,3651.87,6.386032e+10,123.57,1.290459e+10,2.44,2.207200e+09
2019-01-18,3677.99,6.432339e+10,123.65,1.291498e+10,2.52,2.283444e+09


In [6]:
new_df.describe()

,btc_price,btc_market_cap,eth_price,eth_market_cap,eos_price,eos_market_cap
count,2092.000000,2.092000e+03,2092.000000,2.092000e+03,2092.000000,2.092000e+03
mean,2320.828408,3.849109e+10,126.380204,1.233368e+10,1.604152,1.233688e+09
std,3405.036859,5.800273e+10,235.677967,2.306892e+10,3.472965,2.776170e+09
min,68.500000,7.794864e+08,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,327.115000,4.509301e+09,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,608.025000,8.336088e+09,7.335000,6.069067e+08,0.000000,0.000000e+00
75%,3348.935000,5.645490e+10,197.105000,1.914119e+10,0.714197,2.514421e+08
max,19475.800000,3.261873e+11,1397.480000,1.355333e+11,21.640000,1.789341e+10
